In [1]:
from collections import deque

import numpy as np

In [2]:
def parse_input(file):
    with open(file) as file_in:
        grid = file_in.read().splitlines()

    grid = np.array([list(row) for row in grid])

    S_coords = tuple(np.argwhere(grid == 'S')[0].tolist())
    E_coords = tuple(np.argwhere(grid == 'E')[0].tolist())
    wall_coords = set([(x.item(), y.item()) for x, y in np.argwhere(grid == '#')])

    return S_coords, E_coords, wall_coords

In [3]:
def get_best_states(S_coords, E_coords, wall_coords):
    queue = deque([(S_coords, 0)])
    best_states = {S_coords: 0}
    min_score = float('inf')

    while queue:
        (x_current, y_current), score = queue.popleft()

        if score > min_score:
            continue

        if (x_current, y_current) == E_coords:
            min_score = min(min_score, score + 1)
            continue

        for dx, dy in directions:
            x_next, y_next = x_current + dx, y_current + dy 

            if (x_next, y_next) in wall_coords:
                continue

            state = (x_next, y_next)
            if state not in best_states or score + 1 < best_states[state]:
                best_states[state] = score + 1
                queue.append((state, score + 1))

    return best_states

In [4]:
def compute_cheats(best_states, wall_coords):
    cheats = {}

    for x0, y0 in best_states:
        for dx1, dy1 in directions:
            # First move
            x1, y1 = x0 + dx1, y0 + dy1
            if (x1, y1) in wall_coords:
                for dx2, dy2 in directions:
                    # Second move
                    x2, y2 = x1 + dx2, y1 + dy2
                    if (x2, y2) in best_states:
                        gain = best_states[(x2, y2)] - best_states[(x0, y0)] - 2
                        if gain > 0:
                            cheats[((x0, y0), (x2, y2))] = gain

    return cheats

In [9]:
def main1(file):
    S_coords, E_coords, wall_coords = parse_input(file)
    best_states = get_best_states(S_coords, E_coords, wall_coords)
    cheats = compute_cheats(best_states, wall_coords)

    return len([gain for gain in cheats.values() if gain >= 100])

In [10]:
directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]

In [12]:
main1('input.txt')

1363